In [28]:
import numpy as np

# -------------------------------
# Q14 conversion functions
# -------------------------------
def float_to_q14(value):
    """Convert float to Q14 integer"""
    return int(np.round(value * (2**14)))

def q14_to_float(q14_value):
    """Convert Q14 integer back to float"""
    return q14_value / (2**14)

# -------------------------------
# Neuron class with Q14 quantization
# -------------------------------
class Neuron:
    def __init__(self, num_inputs):
        self.weights = np.random.uniform(-1, 1, size=num_inputs)
        self.bias = np.random.uniform(-1, 1)

    def activate(self, inputs):
        self.inputs = inputs
        self.output = self.sigmoid(np.dot(inputs, self.weights) + self.bias)
        return self.output

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    def sigmoid_derivative(self):
        return self.output * (1 - self.output)

    def update_weights(self, delta, learning_rate):
        # Standard gradient update
        self.weights += learning_rate * delta * self.inputs
        self.bias += learning_rate * delta

        # Quantize weights and bias to Q14 after each update
        self.weights = np.array([q14_to_float(float_to_q14(w)) for w in self.weights])
        self.bias = q14_to_float(float_to_q14(self.bias))

# -------------------------------
# Layer class
# -------------------------------
class Layer:
    def __init__(self, num_neurons, num_inputs_per_neuron):
        self.neurons = [Neuron(num_inputs_per_neuron) for _ in range(num_neurons)]

    def forward(self, inputs):
        return np.array([neuron.activate(inputs) for neuron in self.neurons])

    def backward(self, errors, learning_rate):
        deltas = []
        for i, neuron in enumerate(self.neurons):
            # Computes delta for each neuron: error × sigmoid derivative.
            delta = errors[i] * neuron.sigmoid_derivative()
            neuron.update_weights(delta, learning_rate)
            deltas.append(delta)
        return np.dot(np.array([neuron.weights for neuron in self.neurons]).T, deltas)

# -------------------------------
# Neural Network class
# -------------------------------
class NeuralNetwork:
    def __init__(self, layers, learning_rate=0.1, epochs=10000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.layers = []

        for i in range(len(layers) - 1):
            self.layers.append(Layer(layers[i+1], layers[i]))

    def train(self, inputs, outputs):
        for epoch in range(self.epochs):
            total_error = 0
            for x, y in zip(inputs, outputs):
                activations = [x]
                for layer in self.layers:
                    #current layer
                    activations.append(layer.forward(activations[-1]))

                output_errors = y - activations[-1]
                total_error += np.sum(output_errors ** 2)

                errors = output_errors
                for i in reversed(range(len(self.layers))):
                    errors = self.layers[i].backward(errors, self.learning_rate)

            if epoch % 1000 == 0:
                mse = total_error / len(inputs)
                print(f'Epoch {epoch}, MSE: {mse:.6f}')

    def predict(self, inputs):
        activations = inputs
        for layer in self.layers:
            activations = layer.forward(activations)
        return activations

# -------------------------------
# Train XOR network
# -------------------------------
inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
outputs = np.array([[0], [1], [1], [0]])

nn = NeuralNetwork([2, 2, 1], learning_rate=0.1, epochs=10000)
nn.train(inputs, outputs)

# -------------------------------
# Print predictions
# -------------------------------
print("\nPredictions after training:")
for x, y in zip(inputs, outputs):
    pred = nn.predict(x)
    print(f"Input: {x}, Expected: {y[0]}, Predicted: {pred[0]:.4f}, Binary: {int(pred[0] > 0.5)}")

# -------------------------------
# Export VHDL constants (Q14)
# -------------------------------
print("\nVHDL constants for Q14 weights/biases:")
for l_idx, layer in enumerate(nn.layers):
    for n_idx, neuron in enumerate(layer.neurons):
        # Weights
        for w_idx, w in enumerate(neuron.weights):
            q14_val = float_to_q14(w)
            print(f"constant W{l_idx+1}{n_idx}_{w_idx} : signed(31 downto 0) := to_signed({q14_val}, 32);")
        # Bias
        q14_bias = float_to_q14(neuron.bias)
        print(f"constant B{l_idx+1}{n_idx} : signed(31 downto 0) := to_signed({q14_bias}, 32);")


Epoch 0, MSE: 0.334530
Epoch 1000, MSE: 0.244398
Epoch 2000, MSE: 0.205389
Epoch 3000, MSE: 0.109894
Epoch 4000, MSE: 0.023597
Epoch 5000, MSE: 0.010457
Epoch 6000, MSE: 0.006410
Epoch 7000, MSE: 0.004488
Epoch 8000, MSE: 0.003445
Epoch 9000, MSE: 0.002802

Predictions after training:
Input: [0 0], Expected: 0, Predicted: 0.0481, Binary: 0
Input: [0 1], Expected: 1, Predicted: 0.9437, Binary: 1
Input: [1 0], Expected: 1, Predicted: 0.9567, Binary: 1
Input: [1 1], Expected: 0, Predicted: 0.0430, Binary: 0

VHDL constants for Q14 weights/biases:
constant W10_0 : signed(31 downto 0) := to_signed(90628, 32);
constant W10_1 : signed(31 downto 0) := to_signed(-91933, 32);
constant B10 : signed(31 downto 0) := to_signed(-50260, 32);
constant W11_0 : signed(31 downto 0) := to_signed(83372, 32);
constant W11_1 : signed(31 downto 0) := to_signed(-80423, 32);
constant B11 : signed(31 downto 0) := to_signed(39347, 32);
constant W20_0 : signed(31 downto 0) := to_signed(124845, 32);
constant W20_1 :